In [1]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import uniform

## Data

In [6]:
%run ../../functions/vectorize_functions.py

In [7]:
filepath_name = ("..\\..\\..\\data\\mixed_dataset\\train_cleaned.csv")
df_cleaned = pd.read_csv(filepath_name, encoding='utf-8')

In [ ]:
X_train, X_test, y_train, y_test, glove_vectorizer = vectorize_glove(df=df_cleaned, text_column='tweet_cleaned', 
                                                                                 label_column="label")

In [ ]:
with open("svm_glove_vectorizer_opt.pkl", "wb") as file:
    pickle.dump(glove_vectorizer, file)

In [10]:
positive = np.count_nonzero(y_train == 1)
negative = np.count_nonzero(y_train == 0)
print("Train")
print("- Positive:", positive)
print("- Negative:", negative)
print("- Verhältnis:", negative / positive)

positive = np.count_nonzero(y_test == 1)
negative = np.count_nonzero(y_test == 0)
print("Test")
print("- Positive:", positive)
print("- Negative:", negative)
print("- Verhältnis:", negative / positive)

Train
- Positive: 10328
- Negative: 47004
- Verhältnis: 4.551123160340821
Test
- Positive: 4417
- Negative: 20155
- Verhältnis: 4.563051845143763


In [11]:
print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)

print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

X_train shape (57332, 300)
y_train shape (57332,)
X_test shape (24572, 300)
y_test shape (24572,)


## SVM

In [ ]:
# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['sigmoid'],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("..\\..\\..\\data\\mixed_dataset\\test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
X_test = glove_vectorizer.transform(df_test['tweet_cleaned'])
y_test = df_test["label"]

# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_glove_sigmoid_best_model_opt.pkl", "wb") as file:
    pickle.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_glove_sigmoid_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_glove_sigmoid_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_glove_sigmoid_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_glove_sigmoid_classification_report_opt.csv", index=True)


Fitting 2 folds for each of 10 candidates, totalling 20 fits


In [ ]:
# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['linear'],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("..\\..\\..\\data\\mixed_dataset\\test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
X_test = glove_vectorizer.transform(df_test['tweet_cleaned'])
y_test = df_test["label"]

# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_glove_linear_best_model_opt.pkl", "wb") as file:
    pickle.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_glove_linear_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_glove_linear_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_glove_linear_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_glove_linear_classification_report_opt.csv", index=True)

In [ ]:
# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['poly'],
    'degree': [3, 4, 5],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("..\\..\\..\\data\\mixed_dataset\\test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
X_test = glove_vectorizer.transform(df_test['tweet_cleaned'])
y_test = df_test["label"]

# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_glove_poly_best_model_opt.pkl", "wb") as file:
    pickle.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_glove_poly_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_glove_poly_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_glove_poly_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_glove_poly_classification_report_opt.csv", index=True)

In [ ]:
# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['rbf'],
    'degree': [3, 4, 5],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("..\\..\\..\\data\\mixed_dataset\\test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
X_test = glove_vectorizer.transform(df_test['tweet_cleaned'])
y_test = df_test["label"]

# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_glove_rbf_best_model_opt.pkl", "wb") as file:
    pickle.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_glove_rbf_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_glove_rbf_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_glove_rbf_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_glove_rbf_classification_report_opt.csv", index=True)
